# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parameterized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.
* Callback Step - A step that allows users to invoke jobs outside of SageMaker from the Pipeline
* Lambda Step - A step that allows users to invoke a Lambda function from the Pipeline
* Quality Check Step - A step that runs Model Quality or Data Quality checks on a dataset or model. Baselines are created by this step that can be associated with the model when it is added to the Model Registry. 
* Clarify Check Step - A step that uses SageMaker Clarify to find bias and explainability metrics on a model or dataset. 

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
|-- codebuild-buildspec.yml
|-- CONTRIBUTING.md
|-- pipelines
|   |-- Stroke
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- sagemaker-pipelines-project.ipynb
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
|-- codebuild-buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- Stroke
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py

```

For additional subfolders with code and/or artifacts needed by pipeline, they need to be packaged correctly by the `setup.py` file. For example, to package a `pipelines/source` folder,

* Include a `__init__.py` file within the `source` folder.
* Add it to the `setup.py` file's `package_data` like so:

```
...
    packages=setuptools.find_packages(),
    include_package_data=True,
    package_data={"pipelines.my_pipeline.src": ["*.txt"]},
    python_requires=">=3.6",
    install_requires=required_packages,
    extras_require=extras,
...
```

<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:
```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:
```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:
```
`-- tox.ini
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/mm-clarify-pipeline.png)

## Data/Model Quality, Bias, and Model Explainability Checks in SageMaker Pipelines

This notebook introduces two new step types in SageMaker Pipelines - 
* `QualityCheckStep`
* `ClarifyCheckStep`

With these two steps, the pipeline is able to perform baseline calculations that are needed as a standard against which data/model quality issues can be detected (including bias and explainability).

These steps leverage SageMaker pre-built containers:

* `QualityCheckStep` (for Data/Model Quality): [sagemaker-model-monitor-analyzer](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-built-container.html)
* `ClarifyCheckStep` (for Data/Model Bias and Model Explainability): [sagemaker-clarify-processing](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html#clarify-processing-job-configure-container)

The training dataset that you used to trained the model is usually a good baseline dataset. The training dataset data schema and the inference dataset schema should exactly match (the number and order of the features). Note that the prediction/output columns are assumed to be the first columns in the training dataset. From the training dataset, you can ask SageMaker to suggest a set of baseline constraints and generate descriptive statistics to explore the data.

These two new steps will always calculate new baselines using the dataset provided. 

### Drift Check Baselines in the Model Registry

The `RegisterStep` has a new parameter called `drift_check_baselines`. This refers to the baseline files associated with the model. When deployed, these baseline files are used by Model Monitor for Model Quality/Data Quality checks. In addition, these baselines can be used in `QualityCheckStep` and `ClarifyCheckStep` to compare newly trained models against models that have already been registered in the Model Registry. 

### Step Properties

The `QualityCheckStep` has the following properties -

* `CalculatedBaselineStatistics` : The baseline statistics file calculated by the underlying Model Monitor container.
* `CalculatedBaselineConstraints` : The baseline constraints file calculated by the underlying Model Monitor container. 
* `BaselineUsedForDriftCheckStatistics` and `BaselineUsedForDriftCheckConstraints` : These are the two properties used to set `drift_check_baseline` in the Model Registry. The values set in these properties vary depending on the parameters passed to the step. The different behaviors are described in the table below. 
  
The `ClarifyCheckStep` has the following properties - 

* `CalculatedBaselineConstraints` : The baseline constraints file calculated by the underlying Clarify container. 
* `BaselineUsedForDriftCheckConstraints` : This property is used to set `drift_check_baseline` in the Model Registry. The values set in this property will vary depending on the parameters passed to the step. The different behaviors are described in the table below. 

### Getting some constants

We get some constants from the local execution environment.

In [2]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()

# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"StrokeModelPackageGroup"
pipeline_name = f"StrokePipeline"

### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [3]:
from pipelines.stroke.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


SageMaker assumes role: arn:aws:iam::959854082600:role/service-role/AmazonSageMaker-ExecutionRole-20230707T135344.


#### Baseline Calculation and Registration 

Both `QualityCheckStep` and `ClarifyCheckStep` will always calculate a new baseline for a certain check type given required inputs, and the newly calculated baselines are accessible through the step properties `CalculatedBaselineConstraints` and `CalculatedBaselineStatistics`. 

**`ClarifyCheckStep` only generated constraints and not statistics so the properties available on the `ClarifyCheckStep` are `BaselineUsedForDriftCheckConstraints` and `CalculatedBaselineConstraints`**

*Note that baselines suggested by pre-built containers could be very strict, and it is recommended to inspect and adjust suggested baselines before using for drift checks.*

Both `QualityCheckStep` and `ClarifyCheckStep` use two boolean flags `skip_check` and `register_new_baseline` to control their behavior. 

* `skip_check` : This determines if a drift check is executed or not.
* `register_new_baseline` : This determines if the newly calculated baselines (in the step property `CalculatedBaselines`) should be set in the step property `BaselineUsedForDriftCheck`.
* `supplied_baseline_statistics` and `supplied_baseline_constraints` : If skip_check is set to False, baselines can be provided to this step through this parameter. If provided, the step will compare the newly calculated baselines (`CalculatedBaselines`) against those provided here instead of finding the latest baselines from the Model Registry. In the case of `ClarifyCheckStep`, only `supplied_baseline_constraints` is a valid parameter, for `QualityCheckStep`, both parameters are used. 
* `model_package_group_name` : The step will use the `drift_check_baselines` from the latest approved model in the model package group for the drift check. If `supplied_baseline_*` is provided, this field will be ignored.

The first time the pipeline is run, the `skip_check` value should be set to True using the pipeline execution parameters so that new baselines are registered and no drift check is executed.  


In the pipeline we get above, these flags of all the check steps are exposed as pipeline parameters for convenience. The default values of this two flags of all the QualityCheck and ClarifyCheck steps in the pipeline instance we used above are defined as `False`, which indicates in our pipeline runs, we will enable all the drift checks, and previous baseline in each step is assessible through step property `BaselineUsedForDriftCheckConstraints` (and `BaselineUsedForDriftCheckStatistics` if it is a QualtiyCheck step). However, for the initial run we will override them all to `True` since we want to have our initial calculated baselines registered as the baseline for drift check in Model Registry. Details of the flag values and step behaviors will be covered later.

In the RegisterModel step, `BaselineUsedForDriftCheckConstraints` (and `BaselineUsedForDriftCheckStatistics` if it is a `QualtiyCheckStep`) of all QualityCheck and ClarifyCheck steps are registered as `drift_check_baselines` in the new model package created for future drift check purpose, while `CaculatedBaselineConstraints` (and `CalculatedBaselineStatistics` if it is a `QualityCheckStep`) are registered as `ModelMetrics` in the new model package for auditing purpose.  

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [4]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.


{'PipelineArn': 'arn:aws:sagemaker:us-east-2:959854082600:pipeline/StrokePipeline',
 'ResponseMetadata': {'RequestId': 'f5d3e015-72f7-4a56-95f6-ac85671f6daa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f5d3e015-72f7-4a56-95f6-ac85671f6daa',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Thu, 20 Jul 2023 18:41:02 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, values can be passed into these pipeline parameters on starting of the pipeline. Here we override all `skip_check` and `register_new_baseline` to `True` for the initial run as mentioned above.

### Set pipeline parameters


| `skip_check` / `register_new_baseline` | Does step do a drift check?                              | Value of step property `CalculatedBaseline`                                                           | Value of step property `BaselineUsedForDriftCheck`      | Possible Circumstances for this parameter combination|
| -------------------------------------- | ---------------------------------------------------------|------------------------------------------------------------                                           |------------------------------------------------- | -----------------------------------------------------| 
| T / T                                  | No Drift Check.                                          | New baselines calculated by step execution                  | Newly calculated baseline by step execution (value of property `CalculatedBaseline`)                                     | a. Initial run of the pipeline, building the first model version and generate initial baselines. <br>b. Violation detected by the model monitor on endpoint for a particular type of check and the pipeline is triggered for retraining a new model. Skip the check against previous baselines and refresh DriftCheckBaselines with newly calculated baselines in Registry directly.  |
| F / F                                  | Drift Check executed against existing baselines.         | New baselines calculated by step execution                                                            |  Baseline from latest approved model in Model Registry or baseline supplied as step parameter                                    | Regular re-training with checks enabled to get a new model version, but carry over previous baselines as DriftCheckBaselines in Registry for new model version.                                                                                                                                                             |
| F / T                                  | Drift Check executed against existing baselines.         | New baselines calculated by step execution                  | Newly calculated baseline by step execution (value of property `CalculatedBaseline`)                                     | Regular re-training with checks enabled to get a new model version, but refresh DriftCheckBaselines in Registry with newly calculated baselines for the new model version.                                                                                                                                                  |
| T / F                                  | No Drift Check.                                          | New baselines calculated by step execution          | Baseline from latest approved model in Model Registry or baseline supplied as step parameter                                     | Violation detected by the model monitor on endpoint for a particular type of check and the pipeline is triggered for retraining a new model. Skip the check against previous baselines, but carry over previous baselines as DriftCheckBaselines in Registry for new model version.                                                                                                  |

The default parameters are overriden the first time the pipeline is executed

In [6]:
execution = pipeline.start()

### Subsequent runs

Once the baselines have been created for the first time, the default parameters of the pipeline can be used on subsequent runs so that new baselines are compared against old ones if model version approved.

In [ ]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [ ]:
execution.describe()

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [8]:
execution.list_steps()

[{'StepName': 'StrokeErrorPreprocessing',
  'StartTime': datetime.datetime(2023, 7, 20, 18, 43, 13, 288000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 7, 20, 18, 43, 14, 500000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'AttemptCount': 0,
  'FailureReason': 'ClientError: Failed to invoke sagemaker:CreateProcessingJob. Error Details: No S3 objects found under S3 URL "s3://sagemaker-strokeprediction-mlops/data/healthcare-dataset-stroke-data.csv" given in input data source. Please ensure that the bucket exists in the selected region (us-east-2), that objects exist under that S3 prefix, and that the role "arn:aws:iam::959854082600:role/service-role/AmazonSageMaker-ExecutionRole-20230707T135344" has "s3:ListBucket" permissions on bucket "sagemaker-strokeprediction-mlops".',
  'Metadata': {}}]